In [1]:
import os
import tensorflow as tf
import keras
import cv2
import keras.preprocessing.image
import keras_retinanet
import math
from keras_retinanet import models, losses
from keras_retinanet.models.retinanet import retinanet_bbox
from keras_retinanet.models.resnet import resnet50_retinanet
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
#from generator import CustomCSVGenerator
from keras_retinanet.callbacks import RedirectModel
from keras_retinanet.utils.transform import random_transform_generator
#from keras_retinanet.callbacks.eval import Evaluate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from shared import get_session, TrainingType
from callbacks import save_weights_vis, Evaluate
from paths import *

#Create folders
if not os.path.exists(SNAPSHOTS_DIR):
    os.makedirs(SNAPSHOTS_DIR)
    
if not os.path.exists(LOGS_DIR):
    os.makedirs(LOGS_DIR)

/home/madspoderpetersen/deep-traffic/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def create_models(num_classes, weights, freeze_backbone=None):
    modifier = freeze_model if freeze_backbone else None
    model = models.resnet.resnet_retinanet(num_classes, modifier=modifier)
    if weights == "imagenet":
        backbone = models.backbone('resnet50')
        imgnet_weights = backbone.download_imagenet()
        #Load by_name only inputs into the resnet backbone (Tak Thomas)
        model.load_weights(imgnet_weights, skip_mismatch=True, by_name=True)
    elif weights:
        model.load_weights(weights, skip_mismatch=True)

    prediction_model = retinanet_bbox(model=model)
    
    model.trainable = True

    
    for layer in model.layers:
        layer.trainable = True 

    model.compile(
        loss={
            'regression'    : losses.smooth_l1(),
            'classification': losses.focal()
        },
        optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001),
        
    )
    #metrics=['accuracy'] kan inkluderes i model.compile
    
    model.trainable = True
    for layer in model.layers:
        layer.trainable = True 
    
    #model.summary()

    return model, prediction_model

def create_train_generator(batch_size):
    transform_generator = random_transform_generator(
            min_rotation=-0.1,
            max_rotation=0.1,
            min_translation=(-0.1, -0.1),
            max_translation=(0.1, 0.1),
            min_shear=-0.1,
            max_shear=0.1,
            min_scaling=(0.9, 0.9),
            max_scaling=(1.1, 1.1),
            flip_x_chance=0.5,
            #flip_y_chance=0.5,
        )
    
    train_generator = CSVGenerator(
        TRAIN_GROUND_TRUTH_COMBINED,
        CLASS_MAPPINGS_COMBINED,
        batch_size=batch_size,
        transform_generator=transform_generator
    )
    return train_generator

def create_test_generator(batch_size):
    test_generator = CSVGenerator(
        TEST_GROUND_TRUTH_COMBINED,
        CLASS_MAPPINGS_COMBINED,
        shuffle_groups=False,
        batch_size=batch_size,
    )
    return test_generator

def create_callbacks(model, prediction_model, batch_size, test_generator, train_generator, resume):
    callbacks = []

    # save the prediction model
    if resume:
        checkpoint = keras.callbacks.ModelCheckpoint(os.path.join(SNAPSHOTS_DIR, 'custom_resnet50_lolle_resume_imgnet_{epoch:02d}.h5'), verbose=1, period=5)
    else:
        checkpoint = keras.callbacks.ModelCheckpoint(os.path.join(SNAPSHOTS_DIR, 'custom_resnet50_imgnet_{epoch:02d}.h5'), verbose=1, period=5)

    #checkpoint = RedirectModel(checkpoint, model)
    callbacks.append(checkpoint)
    
    #Visualisation of weights
    vis_weights = None
    if resume:
        vis_weights = save_weights_vis(os.path.join(SNAPSHOTS_DIR, 'custom_resnet50_resume_imgnet_{}.png'))
    else:
        vis_weights = save_weights_vis(os.path.join(SNAPSHOTS_DIR, 'custom_resnet50_imgnet_{}.png'))
    callbacks.append(vis_weights)
    
    #Reduce learning rate on plateu (tidligere loss, val_loss)
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, verbose=1, mode='auto', cooldown=0, min_lr=0.001)
    #callbacks.append(lr_scheduler)
    
    #Early stop
    early_stop = EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=1, mode='auto')
    callbacks.append(early_stop)
    
    #For tensorboard
    tensorboard_callb = keras.callbacks.TensorBoard(log_dir=LOGS_DIR, histogram_freq=0, batch_size=batch_size, 
                                                    write_graph=True, write_grads=False, write_images=True, 
                                                    embeddings_freq=0, embeddings_layer_names=None, 
                                                    embeddings_metadata=None)
    callbacks.append(tensorboard_callb)
    
    #Evaluation for the test/train set
    evaluation_test = Evaluate(test_generator, tensorboard=tensorboard_callb, verbose=0, train=False)
    evaluation_test = RedirectModel(evaluation_test, prediction_model)
    callbacks.append(evaluation_test)
    evaluation_train = Evaluate(train_generator, tensorboard=tensorboard_callb, verbose=0, train=True)
    evaluation_train = RedirectModel(evaluation_train, prediction_model)
    callbacks.append(evaluation_train)

    return callbacks

In [ ]:
model = None
training_type = TrainingType.IMGNET
batch_size = 4

keras.backend.tensorflow_backend.set_session(get_session())
train_generator = create_train_generator(batch_size)
test_generator = create_test_generator(batch_size)
resume = False

#From start
if training_type == TrainingType.SCRATCH:
    model, prediction_model = create_models(num_classes=train_generator.num_classes(), weights=None)
#From imagenet
elif training_type == TrainingType.IMGNET:
    model, prediction_model = create_models(num_classes=train_generator.num_classes(), weights="imagenet")
#From earlier 
elif training_type == TrainingType.RESUME:
    #snapshot = os.path.join(SNAPSHOTS_DIR, "custom_resnet50_10.h5")
    snapshot = os.path.join(SNAPSHOTS_DIR, "custom_resnet50_lolle_06.h5")
    resume = True
    model, prediction_model = create_models(num_classes=train_generator.num_classes(), weights=snapshot)

callbacks = create_callbacks(model, prediction_model, batch_size, test_generator, train_generator, resume)

model.trainable = True

# start training
model.fit_generator(
    generator = train_generator,
    #steps_per_epoch = 850,
    steps_per_epoch = math.floor(train_generator.size()/batch_size),
    epochs = 75,
    verbose = 1,
    #workers=6,
    #validation_data=test_generator,
    #validation_steps=math.floor(test_generator.size()/batch_size),
    callbacks = callbacks,
)

Instructions for updating:
Use the retry module or similar alternatives.
Epoch 1/75
151/151 [==============================] - 147s 975ms/step - loss: 5.0200 - regression_loss: 3.8527 - classification_loss: 1.1673
Saving visualised weights of conv1 to file: /home/madspoderpetersen/deep-traffic/snapshots/custom_resnet50_imgnet_1.png
Epoch 2/75
151/151 [==============================] - 135s 895ms/step - loss: 4.2763 - regression_loss: 3.3739 - classification_loss: 0.9024
Saving visualised weights of conv1 to file: /home/madspoderpetersen/deep-traffic/snapshots/custom_resnet50_imgnet_2.png
Epoch 3/75
151/151 [==============================] - 135s 895ms/step - loss: 3.7296 - regression_loss: 3.0850 - classification_loss: 0.6446
Saving visualised weights of conv1 to file: /home/madspoderpetersen/deep-traffic/snapshots/custom_resnet50_imgnet_3.png
Epoch 4/75
151/151 [==============================] - 135s 894ms/step - loss: 3.2008 - regression_loss: 2.6163 - classification_loss: 0.5845
Sav

Epoch 29/75
151/151 [==============================] - 135s 894ms/step - loss: 1.1396 - regression_loss: 0.9323 - classification_loss: 0.2073
Saving visualised weights of conv1 to file: /home/madspoderpetersen/deep-traffic/snapshots/custom_resnet50_imgnet_29.png
Epoch 30/75
151/151 [==============================] - 135s 892ms/step - loss: 1.1156 - regression_loss: 0.9070 - classification_loss: 0.2086

Epoch 00030: saving model to /home/madspoderpetersen/deep-traffic/snapshots/custom_resnet50_imgnet_30.h5
Saving visualised weights of conv1 to file: /home/madspoderpetersen/deep-traffic/snapshots/custom_resnet50_imgnet_30.png
Epoch 31/75
151/151 [==============================] - 135s 893ms/step - loss: 1.0204 - regression_loss: 0.8262 - classification_loss: 0.1941
Saving visualised weights of conv1 to file: /home/madspoderpetersen/deep-traffic/snapshots/custom_resnet50_imgnet_31.png
Epoch 32/75
151/151 [==============================] - 135s 892ms/step - loss: 1.0206 - regression_loss: 

Epoch 58/75
151/151 [==============================] - 134s 889ms/step - loss: 0.7158 - regression_loss: 0.5901 - classification_loss: 0.1256
Saving visualised weights of conv1 to file: /home/madspoderpetersen/deep-traffic/snapshots/custom_resnet50_imgnet_58.png
Epoch 59/75
151/151 [==============================] - 134s 891ms/step - loss: 0.7287 - regression_loss: 0.6018 - classification_loss: 0.1269
Saving visualised weights of conv1 to file: /home/madspoderpetersen/deep-traffic/snapshots/custom_resnet50_imgnet_59.png
Epoch 60/75
151/151 [==============================] - 134s 889ms/step - loss: 0.7308 - regression_loss: 0.6027 - classification_loss: 0.1281

Epoch 00060: saving model to /home/madspoderpetersen/deep-traffic/snapshots/custom_resnet50_imgnet_60.h5
Saving visualised weights of conv1 to file: /home/madspoderpetersen/deep-traffic/snapshots/custom_resnet50_imgnet_60.png
Epoch 61/75
151/151 [==============================] - 135s 891ms/step - loss: 0.7058 - regression_loss: 

In [ ]:
import os
import keras
import numpy as np
from PIL import Image
import tensorflow as tf
from keras_retinanet.models import load_model, backbone, resnet
from keras_retinanet.utils.image import read_image_bgr
from shared import get_session
import matplotlib.pyplot as plt


#Get weights from conv1 layer
weights = np.asarray(model.get_layer('conv1').get_weights())
fig, ax = plt.subplots(figsize=(10, 10),nrows=8, ncols=8)
for i, ax in enumerate(ax.reshape(-1)):
    imgArray = weights[0,:,:,:,i]
    # unity-based normalization https://datascience.stackexchange.com/a/5888
    imgArray = (imgArray - np.min(imgArray)) / (np.max(imgArray) - np.min(imgArray))
    ax.imshow(imgArray)
    ax.axis('off')

    
